In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import scipy.misc
import os
import pickle as pkl
import numpy as np
import argparse
from tqdm import tqdm
from scipy.misc import imresize

In [2]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        # In the simplest case, the output value of the layer with input size (N,Cin,H,W)(N, C_{\text{in}}, H, W)(N,Cin​,H,W) and output (N,Cout,Hout,Wout)(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})(N,Cout​,Hout​,Wout​) 
        self.conv1 = nn.Conv2d(3, 32, 5, 1, 2)
        self.pool1 = nn.MaxPool2d(3,2, ceil_mode = True)
        self.conv2 = nn.Conv2d(32, 16, 5, 1, 2)
        self.pool2 = nn.AvgPool2d(3, 2, ceil_mode = True)
        self.conv3 = nn.Conv2d(16, 32, 5, 1, 2)
        self.pool3 = nn.AvgPool2d(3, 2, ceil_mode = True)
        self.fc1 = nn.Linear(32*4*4, 10)
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        t1 = F.relu(self.pool1(self.conv1(x)))
        t2 = self.pool2(F.relu(self.conv2(t1)))
        t3 = self.pool3(F.relu(self.conv3(t2)))
        x = t3.view(-1, 32 * 4 * 4)
        x = self.fc1(x)
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x


In [3]:
device = torch.device('cpu')
net = torch.load('save_adam_61.pt',map_location=device)
correct = 0

/Users/yaohsiao/anaconda3/envs/cs231n/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/yaohsiao/anaconda3/envs/cs231n/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


as fixed feature extractor

In [4]:
num_ftrs = net.fc1.in_features
print(num_ftrs)
net.fc1 = nn.Linear(num_ftrs, 2)

512


In [5]:
net = net.to(device)

In [52]:
import torch
import torchvision
import torchvision.transforms as transforms

In [53]:
from PIL import Image

In [69]:
# im = Image.open(os.path.join(data_base,"1528642314.jpg"))
# transform = transforms.Compose(
#     [
#     transforms.Resize((32,32)),
#      transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# tm  = transform(im)
# print(tm.shape)

torch.Size([3, 32, 32])


RuntimeError: values not supported on torch.FloatTensor

In [70]:
import pickle
import os
labels = pickle.load(open("mylabel_2.pkl", "rb")) # dictionary
data_base = "../mydata"
img_files = [data for data in os.listdir(data_base) if data.endswith(".jpg")]
valid_img_files = list(set(img_files) & set(labels.keys()))

In [71]:
# tmp = scipy.misc.imread(os.path.join(data_base,"1528642314.jpg"))
# print(type(tmp), tmp.shape)
# img = torch.from_numpy(tmp)
# img = img[np.newaxis, :]
# img = img.permute(0,3,1,2).squeeze(0)
# print(img.shape)

In [72]:
images_data = []
labels_data = []
target_size = (32, 32)
i = 0
for img_name in valid_img_files:
    im = Image.open(os.path.join(data_base,img_name))
    transform = transforms.Compose(
        [
        transforms.Resize((32,32)),
         transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    tm  = transform(im)

# tmp = scipy.misc.imread(os.path.join(data_base,"1528642314.jpg"))
#     tmp = imresize(tmp, target_size)
#     img = torch.from_numpy(tmp)
#     img = img[np.newaxis, :]
#     img = img.permute(0,3,1,2).squeeze(0)
    images_data.append(tm)
    if labels[img_name] == '1':
        labels_data.append(1) 
    else:
        labels_data.append(0)
        
#     img = scipy.misc.imread(os.path.join(self.data_base,img_name))
#     img = imresize(img,target_size)
#     if len(pos_data) >= pos_num and len(neg_data) >= neg_num:
#         break
#     if self.label[img_name] == '1':
#         if len(pos_data) < pos_num :
#             pos_data.append(img)
#             pbar.update(1)
#     else :
#         if len(neg_data) < neg_num :
#             neg_data.append(img)
#             pbar.update(1)
# pbar.close()
# data = pos_data + neg_data
# data = np.stack(data,axis=0)
# label = np.concatenate((np.ones(pos_num),np.zeros(neg_num)))


In [113]:
print(type(images_data))
stacked_images = torch.stack(images_data)
print(stacked_images.shape)
labels_data = torch.from_numpy(np.array(labels_data))
print(len(images_data))
print((images_data[0].shape))

<class 'list'>
torch.Size([411, 3, 32, 32])
411
torch.Size([3, 32, 32])


In [114]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [129]:


for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    i = 0
    while i+1 < 411:
#     for i, data in enumerate(trainloader, 0):
        # get the inputs
#         inputs, labels = data
        inputs = torch.stack([images_data[i], images_data[i+1]])
    
#         print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
#         print(labels_data[i])
        t = torch.ones(2, dtype=torch.int64)
        t[0] = labels_data[i]
        t[1] = labels_data[i+1]
        loss = criterion(outputs, t)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
#         print(i)
        i = i + 2


[1,     1] loss: 0.000
[1,    51] loss: 0.000
[1,   101] loss: 0.001
[1,   151] loss: 0.000
[1,   201] loss: 0.001
[1,   251] loss: 0.001
[1,   301] loss: 0.000
[1,   351] loss: 0.000
[1,   401] loss: 0.000
[2,     1] loss: 0.000
[2,    51] loss: 0.000
[2,   101] loss: 0.001
[2,   151] loss: 0.000
[2,   201] loss: 0.001
[2,   251] loss: 0.001
[2,   301] loss: 0.000
[2,   351] loss: 0.000
[2,   401] loss: 0.000


In [123]:
list(range(0,-1,-3))

[0]